In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ROOT
import landau
from scipy.special import erf
from scipy.integrate import quad
import platform
from multiprocessing import Pool
from tqdm.notebook import tqdm
print(platform.python_version())

In [ ]:
dosave=False
plt.rcParams.update({'font.size': 14})

In [ ]:
# constants -- units MeV, cm, s
# MeV * cm
# hbarc = 1.973269804 * 1e-11
# cm / s
# c = 2.998 * 1e10

# alpha = 1. / 137.
Mmuon = 105.6
Melec = 0.5110
Relec = 2.817940 * 1e-13

# energy scale [MeV]
muonE = 1_000 # 140 + Mmuon
# motion -- beta
beta = np.sqrt(1 - (Mmuon * Mmuon) / (muonE * muonE))
# motion -- gamma
gamma = muonE / Mmuon
# motion -- (gamma * beta)^2
gammaB2 = gamma*gamma*beta*beta

# electron ionization energy [MeV]
I0 = 188.0 * 1e-6

# maximum energy transfer [Mev]
# maxE = 2 * Melec * gammaB2
maxE = 2 * Melec * gammaB2 / \
    ( 1 + 2 * gamma * Melec / Mmuon + (Melec/Mmuon)**2)

# get the density of argon 
# density -- g / mL
LAr_density_gmL = 1.396
# molar mass -- g / mol
Ar_molar_mass = 39.9623
# avogadro number
mole = 6.0221409*1e23
# charge number / atomic mass (Z / A)
# 99.6% of LAr is Ar40
Ar_ZA = 18. / Ar_molar_mass

# electron number density (N / cm^3)
density = Ar_ZA * mole * LAr_density_gmL

# wirep (cm)
wirep = 0.3

# MeV cm^2 / mol
K = 4*np.pi*mole*Relec**2*Melec # 0.307075

# outputdir = "/home/grayputnam/Work/Summer2020/July13BetheBlochBarkas/"
zeta = (K/2.)*Ar_ZA*(1./beta**2) * LAr_density_gmL / maxE

def f_kappa(thickness):
    return zeta * thickness

mean_dEdx = LAr_density_gmL * K * Ar_ZA * (1 / (beta * beta)) * (\
        (1./2)*np.log( maxE*2 * gammaB2*Melec / (I0**2)) \
        - beta * beta)

def f_mpv(thickness):
    kappa = f_kappa(thickness)*maxE
    j=0.200
    return (kappa/thickness)*\
    (np.log(2 * Melec * gammaB2 / I0) + np.log(kappa/I0) + j - beta*beta)

def smeared_dep(x0, w, sigma):
    if sigma < 1e-4:
        return 1*(np.abs(x0) < w/2)
    return (1./2)*(erf((w/2+x0)/(np.sqrt(2)*sigma)) +\
                   erf((w/2-x0)/(np.sqrt(2)*sigma)))

def f_thickness(sigma, a=wirep):
    return a*np.exp(quad(lambda x: -smeared_dep(x, a, sigma) * np.log(smeared_dep(x, a, sigma))/a, 
                       -(a/2) - 5*sigma, (a/2) + 5*sigma)[0])

Dtransverse = 5.85e-3 # cm^2/ms
def smearing(driftT):
    return np.sqrt(2*Dtransverse*driftT)

In [ ]:
# ArgoNeuT
f_mpv(0.4), f_mpv(f_thickness(smearing(0.295), 0.4))

In [ ]:
# MicroBooNE
f_mpv(0.3), f_mpv(f_thickness(smearing(2.33), 0.3))

In [ ]:
# ICARUS
f_mpv(0.3), f_mpv(f_thickness(smearing(0.96), 0.3))

In [ ]:
# SBND
f_mpv(0.3), f_mpv(f_thickness(smearing(1.28), 0.3))

In [ ]:
# DUNE-FD
f_mpv(0.47), f_mpv(f_thickness(smearing(2.2), 0.47))

In [ ]:
# Plot thickness dependance
sigmas = np.linspace(0, 2, 21)
thicks = [f_thickness(s, 1) for s in sigmas]

plt.plot(sigmas, thicks, label="Step Function(a)\n $\\circledast$ Gaussian($\\sigma$)")
plt.plot(sigmas, np.sqrt(2*np.pi*np.e)*sigmas, linestyle=":", color="black", label="Large $\\sigma$ Limit")
# plt.gca().set_xticklabels(["%.1fa" % l for l in plt.xticks()[0]])
plt.xlabel("Gaussian Width ($\\sigma$) / Wire Pitch ($a$)")
plt.ylabel("Thickness ($\\mathscr{t}$) / Wire Pitch ($a$)")

plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "step_gaussian_thickness.png")

In [ ]:
Nmc = 1000
Nsample = 1000
Nmciter = 1000

dx = 0.01
kappa = f_kappa(dx)
xs = np.linspace(-100, 100, int(2e5))
pdf = [ROOT.TMath.VavilovI(x, kappa, beta**2) for x in xs]

In [ ]:
ROOT.gInterpreter.Declare("""
    double vavilov(double kappa, double beta2) {
        return ROOT::Math::Vavilov(kappa, beta2);
    }
""")

In [ ]:
kappa

In [ ]:
# smearings = np.linspace(0, 1., 51)
smearings = np.linspace(0, 0.15, 16)

mpvs = []
mpv_errs = []

def get_esum(i):
    dE = Es[np.searchsorted(pdE, np.random.random(Nsample*Nmc))]
    xs = np.linspace(-pitch*(Nsample/2), pitch*(Nsample/2), Nsample+1)
    xcenter = (xs[1:] + xs[:-1]) / 2.

    thisEsum = np.sum(dE.reshape(Nmc, Nsample)*smeared_dep(xcenter, wirep, smearing), axis=-1)

    return thisEsum

for smearing in smearings:
    plt.figure()
    
    Esums = []
    with Pool(12) as p:
        for esum in tqdm(p.imap_unordered(get_esum, range(Nmciter), 5), total=Nmciter):
            Esums.append(esum)
        
    Esum = np.concatenate(Esums)

    bins = np.linspace(1., 5.0, 121)
    N, _, _ = plt.hist(Esum/wirep, bins=bins)

    bin_centers = (bins[:-1] + bins[1:]) / 2.

    p0 = [1.7, 0.1, 0.001, np.max(N)*40]
    bounds = ([1.5, 0.005, 0.0001, np.max(N)*40/100], 
              [2., 1., 1., np.max(N)*40*100])
    
    p0 = [2.15, 0.1, np.max(N)*40]
    bounds = ([1.5, 0.005, np.max(N)*40/100], 
          [2.5, 5, np.max(N)*40*100])


    maxbin = np.argmax(N)
    when_fit = np.abs(np.array((range(len(bin_centers)))) - maxbin) < 50

    popt, perr = curve_fit(landau_f, bin_centers[when_fit], N[when_fit],
                          p0=p0, maxfev=100_000, sigma=np.maximum(np.sqrt(N), 1)[when_fit], bounds=bounds)

    print(popt)
    plt.plot(bin_centers, landau_f(bin_centers, *popt))
    m, m_err = get_mpv(popt, perr)
    
    plt.xlabel("Energy Loss [MeV/cm]")
    plt.ylabel("Entries")
    plt.text(0.35, 0.5, "M.P.V. =\n   %.3f +/- %.3f MeV/cm" % (m, m_err), fontsize=16, transform=plt.gca().transAxes)
    
    mpvs.append(m)
    mpv_errs.append(m_err)
    plt.xlim([1, 4])
    plt.title("Smearing: %.2fcm" % smearing)
    plt.tight_layout()
    
    dEdx = Esum/wirep

    if dosave: plt.savefig((savedir + "eloss_dist_smearing%.2f_E%.0f" % (smearing, muonE)).replace(".", "_") + ".png")
